In [1]:
import requests
import json
import pandas as pd

예시 도서

- 이방인 : 9788937443848
- 해리포터 : 9788956054421
- 지구 끝의 온실 : 9791191824001
- 날씨가 좋으면 찾아가겠어요 : 9788952791160

아래는 '날씨가 좋으면 찾아가겠어요'로 수행한 예시

#### 도서 정보 불러오기

In [2]:
rest_api_key = "41d651c93152d5ec054dc828cacfa671"
url = "https://dapi.kakao.com/v3/search/book"
header = {"authorization": "KakaoAK "+rest_api_key}

In [3]:
#궁금한 도서의 isbn 입력
querynum = {"query": "9788952791160"}

In [4]:
#도서 정보 불러오기
response = requests.get(url, headers=header, params = querynum)
content = response.text
책정보 = json.loads(content)['documents'][0]

#### 도서 정보 확인

- title : 도서 제목
- contents : 도서 소개
- url : 도서 상세 URL
- isbn : ISBN10(10자리) 또는 ISBN13(13자리) 형식의 국제 표준 도서번호(International Standard Book Number) 중 하나 이상 포함
- datetime : 도서 출판날짜, ISO 8601 형식 ([YYYY]-[MM]-[DD]T[hh]:[mm]:[ss].000+[tz])
- authors : 도서 저자 리스트
- publisher : 도서 출판사
- translators : 도서 번역자 리스트
- price : 도서 정가
- sale_price : 도서 판매가
- thumbnail : 도서 표지 미리보기 URL
- status : 도서 판매 상태 정보 (정상, 품절, 절판 등)

In [193]:
책정보

{'authors': ['이도우'],
 'contents': '《사서함 110호의 우편물》의 저자 이도우가 《잠옷을 입으렴》 이후 6년 만에 펴낸 장편소설 『날씨가 좋으면 찾아가겠어요』. 유년 시절 산에 살았던 어떤 소년, 인생 첫 단골 서점, 미로 같았던 반년간의 여름날, 새벽이 가까울 무렵 올라오는 야행성 사람들의 SNS 글 등 때로는 스쳐 간, 더 많이는 온전히 남은 삶의 여러 조각을 모아 만든 작품으로 서로에게 많이 미안한 이들이 오랫동안 하지 못했던 말들을 세월이 흐른 후 비로소 용기 내어 전하는',
 'datetime': '2018-06-28T00:00:00.000+09:00',
 'isbn': '8952791169 9788952791160',
 'price': 13800,
 'publisher': '시공사',
 'sale_price': 12420,
 'status': '정상판매',
 'thumbnail': 'https://search1.kakaocdn.net/thumb/R120x174.q85/?fname=http%3A%2F%2Ft1.daumcdn.net%2Flbook%2Fimage%2F648446%3Ftimestamp%3D20230901160933',
 'title': '날씨가 좋으면 찾아가겠어요',
 'translators': [],
 'url': 'https://search.daum.net/search?w=bookpage&bookId=648446&q=%EB%82%A0%EC%94%A8%EA%B0%80+%EC%A2%8B%EC%9C%BC%EB%A9%B4+%EC%B0%BE%EC%95%84%EA%B0%80%EA%B2%A0%EC%96%B4%EC%9A%94'}

#### 필요 정보만 추출

title / contents / isbn / authors / publisher

In [159]:
book = pd.DataFrame({'title': 책정보['title'],
              'isbn': 책정보['isbn'],
              'authors': 책정보['authors'],
              'publisher': 책정보['publisher']})

In [160]:
book

,title,isbn,authors,publisher
0,날씨가 좋으면 찾아가겠어요,8952791169 9788952791160,이도우,시공사


#### url로 들어가서 책 정보 수집

: 책소개, 책 속으로, 출판사 서평

In [162]:
target_url = 책정보['url']

In [167]:
#!pip install selenium

In [166]:
from selenium import webdriver
from selenium.webdriver.common.by import By

# 해당 url을 크롬 브라우저에 띄운다.
driver = webdriver.Chrome()
driver.get(target_url)

In [183]:
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[2]/div[3]/a')
    botton.click()
except :
    pass

책소개 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[2]/p')
책소개.text

'《사서함 110호의 우편물》의 저자 이도우가 《잠옷을 입으렴》 이후 6년 만에 펴낸 장편소설 『날씨가 좋으면 찾아가겠어요』. 유년 시절 산에 살았던 어떤 소년, 인생 첫 단골 서점, 미로 같았던 반년간의 여름날, 새벽이 가까울 무렵 올라오는 야행성 사람들의 SNS 글 등 때로는 스쳐 간, 더 많이는 온전히 남은 삶의 여러 조각을 모아 만든 작품으로 서로에게 많이 미안한 이들이 오랫동안 하지 못했던 말들을 세월이 흐른 후 비로소 용기 내어 전하는 이야기를 담고 있다.\n\n\n\n미대입시학원에서 그림을 가르치던 해원은 학생과의 불화를 계기로 일을 그만두고 펜션을 운영하는 이모 곁에서 한동안 지내기로 한다. 열다섯 살 그 일 이후로 사람에게 기대한 적이 없었던 해원은 언젠가부터 사람을 그리는 것이 싫어 인물화를 그리지 않는다. 한편 노부부가 살던 낡은 기와집을 작은 서점 굿나잇책방으로 바꾸어 운영하고 있는 은섭은 자신의 서점을 기웃거리는 해원을 보고 놀란다. 삼 년 전 은섭에게 겨울 들판에 뒹구는 ‘마시멜로’의 진짜 이름이 뭐냐고 묻던 이웃집 그녀다.\n\n\n\n이모를 통해 그 낯선 서점의 책방지기가 옆집 사는 은섭이라는 것을 알게 된 해원은 굿나잇책방으로 향한다. 그리고 그녀가 머물 겨울 동안 책방 매니저 아르바이트를 하기로 한다. 어린 시절 타인에게 신경을 곤두세우며 살았다고 생각했지만 알고 보니 자기에만 매몰되어 있었던 해원은 은섭과 같은 중고등학교를 나왔지만 그를 잘 모른다. 해원이 알고 있던 것보다 은섭이 그녀의 인생 어떤 페이지에 더 많이 등장했었다는 사실도…….'

In [184]:
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[5]/div[3]/a')
    botton.click()
except :
    pass

책속으로 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[5]/p')
책속으로.text

'세상에 흩어져 잠자리에 드는 굿나잇클럽 여러분도 잘 자요. 겨울 들판의 마시멜로를 보면 강원도 어딘가에서 바보 같은 대답을 한 인간이 있다는 걸 기억해주세요. 내년 겨울에 또 물어봐, 자꾸자꾸 대답할게 같은 멍청한 소리를 하는 남자가 있다는 것을. 여러분은 더 잘할 수 있을 겁니다, 저보다는.\n그럼 굿나잇.\n이 마을은 서리가 내려 환합니다, 로저.\n-32쪽 중에서\n\n그의 사랑은… 눈송이 같을 거라고 해원은 생각했다. 하나둘 흩날려 떨어질 땐 아무런 무게도 부담도 느껴지지 않다가, 어느 순간 마을을 덮고 지붕을 무너뜨리듯 빠져나오기 힘든 부피로 다가올 것만 같다고. 그만두려면 지금 그래야 한다 싶었지만 그의 외로워 보이는 눈빛에서 피할 수가 없고, 그건 그도 마찬가지인 것 같았다.\n-198쪽 중에서\n\n알고 보면 사람들은 참 이상하고도 신기한 존재였다. 꽃은 타고난 대로 피어나고 질 뿐인데 그걸 몹시 사랑하고 예뻐하고… 꽃말까지 지어 붙인다. 의미를 담아 주고받으며, 말하지 않아도 마음이 전해지길 바라기도 한다. 꽃들은 무심하고, 의미는 그들이 알 바가 아니었다. 그저 계절 따라 피었다 지고 사람들만 울고 웃는다. 어느새 봄기운이 완연했다.\n-407쪽 중에서'

In [185]:
try :
    botton = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[6]/div[3]/a')
    botton.click()
except :
    pass

서평 = driver.find_element(By.XPATH, '//*[@id="tabContent"]/div[1]/div[6]/p')
서평.text

'“겨울이 좋은 이유는 그저 한 가지\n내 창을 가리던 나뭇잎들이 떨어져\n건너편 당신의 창이 보인다는 것”\n\n해원은 그림을 가르치던 일을 그만두고 호두하우스 펜션을 운영하는 이모 곁에서 한동안 지내기로 한다. 노부부가 살던 기와집이 작은 서점 ‘굿나잇책방’으로 바뀐 것을 보고 의아해한다. ‘이 시골에 서점…?’\n논두렁 스케이트장에서 일하던 은섭은 그의 닫힌 책방을 기웃거리는 해원을 보고 멈칫 놀란다. 언젠가 그에게 무심히 겨울 들판의 마시멜로 이름을 묻던 이웃집 그녀가 돌아왔으니까….\n\n■ 작품 소개\n\n6년의 기다림 끝에 만나는\n《사서함 110호의 우편물》 이도우의 최신 장편소설\n\n“네 사랑이 무사하기를, 내 사랑도 무사하니까. 세상의 모든 사랑이 무사하기를”이라는 문장으로 회자되며 25만 독자들의 인생책으로 뜨겁게 사랑받고 있는 롱 스테디셀러 《사서함 110호의 우편물》의 이도우 작가가 신작으로 찾아왔다. 《잠옷을 입으렴》 이후 6년 만에 발표한 장편소설 《날씨가 좋으면 찾아가겠어요》는 시골 마을의 낡은 기와집에 자리한 작은 서점 ‘굿나잇책방’을 중심으로 한 용서와 치유 그리고 사랑에 대한 이야기다. 슬픔을 누그러뜨리는 뜨거운 차처럼 먹고사는 일 때문에 혹은 사람 때문에 날이 선 마음을 누그러뜨려주는 이 작품은, 시간과 세대를 뛰어넘어 독자의 마음을 두드릴 작가의 또 다른 대표작이 되리라는 예감을 준다.\n유년 시절 산에 살았던 어떤 소년, 인생 첫 단골 서점, 미로 같았던 반년간의 여름날, 새벽이 가까울 무렵 올라오는 야행성 사람들의 SNS 글 등 때로는 스쳐 간, 더 많이는 온전히 남은 삶의 여러 조각을 모아 만든 소설이라고 작가 스스로 밝힌 《날씨가 좋으면 찾아가겠어요》는 서로에게 많이 미안한 이들이 오랫동안 하지 못했던 말들을 세월이 흐른 후 비로소 용기 내어 전하는 이야기를 담았다고 한다. “다음에 만나야지”, “날이 좀 풀리면 얼굴 한번 보자”와 같이 그저 말로만 끝나는 순간들. 날씨가 좋아지면, 준비가 되면, 때가 되면, 성공

In [186]:
book['책소개'] = 책소개.text
book['책속으로'] = 책속으로.text
book['서평'] = 서평.text

#### 최종 책 정보

`book` : 제목, isbn, 저자, 출판사, 책소개, 책속으로, 서평(출판사 서평)

In [187]:
book

,title,isbn,authors,publisher,책소개,책속으로,서평
0,날씨가 좋으면 찾아가겠어요,8952791169 9788952791160,이도우,시공사,《사서함 110호의 우편물》의 저자 이도우가 《잠옷을 입으렴》 이후 6년 만에 펴낸...,세상에 흩어져 잠자리에 드는 굿나잇클럽 여러분도 잘 자요. 겨울 들판의 마시멜로를 ...,“겨울이 좋은 이유는 그저 한 가지\n내 창을 가리던 나뭇잎들이 떨어져\n건너편 당...
